In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.externals import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
  
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1hRQAAIQX8Llj78w4RtTu92b-Lnb6bNkw'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_clean.csv')#500W time signal open_channels
dtrain = pd.read_csv('train_clean.csv')
print(dtrain)

             time    signal  open_channels
0          0.0001 -2.760000              0
1          0.0002 -2.855700              0
2          0.0003 -2.407400              0
3          0.0004 -3.140400              0
4          0.0005 -3.152500              0
...           ...       ...            ...
4999995  499.9996  2.919274              7
4999996  499.9997  2.697906              7
4999997  499.9998  4.516337              8
4999998  499.9999  5.639669              9
4999999  500.0000  5.379200              9

[5000000 rows x 3 columns]


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '10E4nJR2oOl2bjj6_BGW22Ioru8O_ivhy'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_clean.csv')#200W time signal
dtest = pd.read_csv('test_clean.csv')
print(dtest)

             time    signal
0        500.0001 -2.649831
1        500.0002 -2.849463
2        500.0003 -2.860094
3        500.0004 -2.435126
4        500.0005 -2.615657
...           ...       ...
1999995  699.9996 -2.909200
1999996  699.9997 -2.742200
1999997  699.9998 -2.828500
1999998  699.9999 -2.909200
1999999  700.0000 -2.742200

[2000000 rows x 2 columns]
             time    signal
0        500.0001 -2.649831
1        500.0002 -2.849463
2        500.0003 -2.860094
3        500.0004 -2.435126
4        500.0005 -2.615657
...           ...       ...
1999995  699.9996 -2.909200
1999996  699.9997 -2.742200
1999997  699.9998 -2.828500
1999998  699.9999 -2.909200
1999999  700.0000 -2.742200

[2000000 rows x 2 columns]


In [ ]:
dtrain.describe()

,time,signal,open_channels
count,5.000000e+06,5.000000e+06,5.000000e+06
mean,2.500001e+02,8.261117e-02,2.726043e+00
std,1.443376e+02,2.487782e+00,2.665051e+00
min,1.000000e-04,-5.796500e+00,0.000000e+00
25%,1.250001e+02,-2.101000e+00,1.000000e+00
50%,2.500000e+02,-3.163000e-01,2.000000e+00
75%,3.750000e+02,1.976200e+00,4.000000e+00
max,5.000000e+02,8.613100e+00,1.000000e+01


In [ ]:
#train 的 rolling統計值
window_sizes = [10, 50, 100, 1000]
for window in window_sizes:
    dtrain['rolling_mean_'+str(window)]=dtrain['signal'].rolling(window=window).mean()
    dtrain['rolling_var_'+str(window)]=dtrain['signal'].rolling(window=window).var()
    dtrain['rolling_std_'+str(window)]=dtrain['signal'].rolling(window=window).std()
    dtrain['rolling_min_'+str(window)]=dtrain['signal'].rolling(window=window).min()
    dtrain['rolling_max_'+str(window)]=dtrain['signal'].rolling(window=window).max()
    
    #最小值最大值正規化(Min-Max scaler):(Xi-min)/(max-min)，縮放[0,1]間
    a =(dtrain['signal'] - dtrain['rolling_min_' + str(window)])/(dtrain['rolling_max_' + str(window)] - dtrain['rolling_min_' + str(window)])
    #無條件進位
    dtrain["norm_" + str(window)] = a * (np.floor(dtrain['rolling_max_' + str(window)]) - np.ceil(dtrain['rolling_min_' + str(window)]))

dtrain = dtrain.replace([np.inf, -np.inf], np.nan)    
dtrain.fillna(0, inplace=True)

In [ ]:
#test 的 rolling統計值
window_sizes = [10, 50, 100, 1000]
for window in window_sizes:
    dtest['rolling_mean_'+str(window)]=dtest['signal'].rolling(window=window).mean()
    dtest['rolling_var_'+str(window)]=dtest['signal'].rolling(window=window).var()
    dtest['rolling_std_'+str(window)]=dtest['signal'].rolling(window=window).std()
    dtest['rolling_min_'+str(window)]=dtest['signal'].rolling(window=window).min()
    dtest['rolling_max_'+str(window)]=dtest['signal'].rolling(window=window).max()
    #最小值最大值正規化(Min-Max scaler):(Xi-min)/(max-min)，縮放[0,1]間
    a =(dtest['signal'] - dtest['rolling_min_' + str(window)])/(dtest['rolling_max_' + str(window)] - dtest['rolling_min_' + str(window)])
    #無條件進位
    dtest["norm_" + str(window)] = a * (np.floor(dtest['rolling_max_' + str(window)]) - np.ceil(dtest['rolling_min_' + str(window)]))
dtest = dtest.replace([np.inf, -np.inf], np.nan)    
dtest.fillna(0, inplace=True)

In [ ]:
import gc
def features(df):
    #更新index 不然會出奇怪的錯
    df = df.sort_values(by=['time']).reset_index(drop=True)

    #0~4999999
    df.index = ((df.time * 10_000) - 1).values 
    
    #500W/2.5W=分200batchs，0~199
    df['batch'] = df.index // 25_000 

    #0 ~ 24999
    df['batch_index'] = df.index  - (df.batch * 25_000) 

    #25000/2500=10 , 0.0 ~ 9.0
    df['batch_slices'] = df['batch_index']  // 2500 

    # 0.0_0.0 ~ 199.0_9.0 , zfill指定長度為3，加r是為了宣告這是raw string不要亂轉
    df['batch_slices2'] = df.apply(lambda r: '_'.join([str(r['batch']).zfill(3), str(r['batch_slices']).zfill(3)]), axis=1)


    
    for c in ['batch','batch_slices2']:
        #字典
        d = {}

        #batchs的mean median max min std
        d['mean'+c] = df.groupby([c])['signal'].mean()
        d['median'+c] = df.groupby([c])['signal'].median()
        d['max'+c] = df.groupby([c])['signal'].max()
        d['min'+c] = df.groupby([c])['signal'].min()
        d['std'+c] = df.groupby([c])['signal'].std()

        #absolute change:diff=離散差，後-前，signal的diff的絕對值的mean，每batch取一次
        d['mean_abs_chg'+c] = df.groupby([c])['signal'].apply(lambda x: np.mean(np.abs(np.diff(x))))

        #取絕對值後的max min 計算出range max/min max到min平均
        d['abs_max'+c] = df.groupby([c])['signal'].apply(lambda x: np.max(np.abs(x)))
        d['abs_min'+c] = df.groupby([c])['signal'].apply(lambda x: np.min(np.abs(x)))
        d['range'+c] = d['max'+c] - d['min'+c]
        d['maxtomin'+c] = d['max'+c] / d['min'+c]
        d['abs_avg'+c] = (d['abs_min'+c] + d['abs_max'+c]) / 2

        for v in d:
            #將字典d mapping成dataframe
            df[v] = df[c].map(d[v].to_dict())

    
    # add shifts_1

    #加一條全0的record+選取除了最後一筆signal的values並將其替換
    df['signal_shift_+1'] = [0,] + list(df['signal'].values[:-1])
    df['signal_shift_-1'] = list(df['signal'].values[1:]) + [0]
    for i in df[df['batch_index']==0].index:
        df['signal_shift_+1'][i] = np.nan
    for i in df[df['batch_index']==49999].index:
        df['signal_shift_-1'][i] = np.nan
    
    # add shifts_2 - my upgrade
    df['signal_shift_+2'] = [0,] + [1,] + list(df['signal'].values[:-2])
    df['signal_shift_-2'] = list(df['signal'].values[2:]) + [0] + [1]
    for i in df[df['batch_index']==0].index:
        df['signal_shift_+2'][i] = np.nan
    for i in df[df['batch_index']==1].index:
        df['signal_shift_+2'][i] = np.nan
    for i in df[df['batch_index']==49999].index:
        df['signal_shift_-2'][i] = np.nan
    for i in df[df['batch_index']==49998].index:
        df['signal_shift_-2'][i] = np.nan
    
    df = df.drop(columns=['batch', 'batch_index', 'batch_slices', 'batch_slices2'])

        
    df = df.replace([np.inf, -np.inf], np.nan)    
    df.fillna(0, inplace=True)
    gc.collect()
    return df



In [ ]:
#用xgbregressor填nan?
dtrain = features(dtrain)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dtest = features(dtest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dtrain

,time,signal,open_channels,rolling_mean_10,rolling_var_10,rolling_std_10,rolling_min_10,rolling_max_10,norm_10,rolling_mean_50,rolling_var_50,rolling_std_50,rolling_min_50,rolling_max_50,norm_50,rolling_mean_100,rolling_var_100,rolling_std_100,rolling_min_100,rolling_max_100,norm_100,rolling_mean_1000,rolling_var_1000,rolling_std_1000,rolling_min_1000,rolling_max_1000,norm_1000,meanbatch,medianbatch,maxbatch,minbatch,stdbatch,mean_abs_chgbatch,abs_maxbatch,abs_minbatch,rangebatch,maxtominbatch,abs_avgbatch,meanbatch_slices2,medianbatch_slices2,maxbatch_slices2,minbatch_slices2,stdbatch_slices2,mean_abs_chgbatch_slices2,abs_maxbatch_slices2,abs_minbatch_slices2,rangebatch_slices2,maxtominbatch_slices2,abs_avgbatch_slices2,signal_shift_+1,signal_shift_-1,signal_shift_+2,signal_shift_-2
0.0,0.0001,-2.760000,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.683757,-2.692300,-0.765100,-3.655100,0.272890,0.274253,3.655100,0.765100,2.89000,0.209324,2.210100,-2.693769,-2.694000,-1.902900,-3.466900,0.244439,0.268949,3.466900,1.902900,1.564000,0.548877,2.684900,0.000000,-2.855700,0.000000,-2.407400
1.0,0.0002,-2.855700,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.683757,-2.692300,-0.765100,-3.655100,0.272890,0.274253,3.655100,0.765100,2.89000,0.209324,2.210100,-2.693769,-2.694000,-1.902900,-3.466900,0.244439,0.268949,3.466900,1.902900,1.564000,0.548877,2.684900,-2.760000,-2.407400,0.000000,-3.140400
2.0,0.0003,-2.407400,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.683757,-2.692300,-0.765100,-3.655100,0.272890,0.274253,3.655100,0.765100,2.89000,0.209324,2.210100,-2.693769,-2.694000,-1.902900,-3.466900,0.244439,0.268949,3.466900,1.902900,1.564000,0.548877,2.684900,-2.855700,-3.140400,-2.760000,-3.152500
3.0,0.0004,-3.140400,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.683757,-2.692300,-0.765100,-3.655100,0.272890,0.274253,3.655100,0.765100,2.89000,0.209324,2.210100,-2.693769,-2.694000,-1.902900,-3.466900,0.244439,0.268949,3.466900,1.902900,1.564000,0.548877,2.684900,-2.407400,-3.152500,-2.855700,-2.641800
4.0,0.0005,-3.152500,0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.683757,-2.692300,-0.765100,-3.655100,0.272890,0.274253,3.655100,0.765100,2.89000,0.209324,2.210100,-2.693769,-2.694000,-1.902900,-3.466900,0.244439,0.268949,3.466900,1.902900,1.564000,0.548877,2.684900,-3.140400,-2.641800,-2.407400,-2.699300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995.0,499.9996,2.919274,7,3.860933,0.670170,0.818639,2.34818,5.116992,0.412519,4.671041,1.444595,1.201913,1.500529,7.166932,1.251892,4.268307,1.362005,1.167050,1.386867,7.166932,1.325597,3.723650,2.201502,1.483746,-1.285073,7.787752,3.707200,3.402610,3.428574,8.166912,-3.829699,1.799165,1.070229,8.166912,0.000141,11.99661,-2.132521,4.083526,3.307970,3.334694,7.961413,-3.000673,1.738972,1.123281,7.961413,0.003232,10.962086,-2.653209,3.982322,4.371143,2.697906,4.191312,4.516337
4999996.0,499.9997,2.697906,7,3.619024,0.580143,0.761671,2.34818,4.508017,0.161922,4.647356,

In [ ]:
dtest

,time,signal,rolling_mean_10,rolling_var_10,rolling_std_10,rolling_min_10,rolling_max_10,norm_10,rolling_mean_50,rolling_var_50,rolling_std_50,rolling_min_50,rolling_max_50,norm_50,rolling_mean_100,rolling_var_100,rolling_std_100,rolling_min_100,rolling_max_100,norm_100,rolling_mean_1000,rolling_var_1000,rolling_std_1000,rolling_min_1000,rolling_max_1000,norm_1000,meanbatch,medianbatch,maxbatch,minbatch,stdbatch,mean_abs_chgbatch,abs_maxbatch,abs_minbatch,rangebatch,maxtominbatch,abs_avgbatch,meanbatch_slices2,medianbatch_slices2,maxbatch_slices2,minbatch_slices2,stdbatch_slices2,mean_abs_chgbatch_slices2,abs_maxbatch_slices2,abs_minbatch_slices2,rangebatch_slices2,maxtominbatch_slices2,abs_avgbatch_slices2,signal_shift_+1,signal_shift_-1,signal_shift_+2,signal_shift_-2
5000000.0,500.0001,-2.649831,0.00000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,-2.661265,-2.68356,-0.884686,-3.817629,0.309878,0.277272,3.817629,0.884686,2.932943,0.231737,2.351157,-2.693608,-2.690864,-1.847187,-3.403764,0.242972,0.266850,3.403764,1.847187,1.556577,0.542689,2.625475,0.000000,-2.849463,0.000000,-2.860094
5000001.0,500.0002,-2.849463,0.00000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,-2.661265,-2.68356,-0.884686,-3.817629,0.309878,0.277272,3.817629,0.884686,2.932943,0.231737,2.351157,-2.693608,-2.690864,-1.847187,-3.403764,0.242972,0.266850,3.403764,1.847187,1.556577,0.542689,2.625475,-2.649831,-2.860094,0.000000,-2.435126
5000002.0,500.0003,-2.860094,0.00000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,-2.661265,-2.68356,-0.884686,-3.817629,0.309878,0.277272,3.817629,0.884686,2.932943,0.231737,2.351157,-2.693608,-2.690864,-1.847187,-3.403764,0.242972,0.266850,3.403764,1.847187,1.556577,0.542689,2.625475,-2.849463,-2.435126,-2.649831,-2.615657
5000003.0,500.0004,-2.435126,0.00000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,-2.661265,-2.68356,-0.884686,-3.817629,0.309878,0.277272,3.817629,0.884686,2.932943,0.231737,2.351157,-2.693608,-2.690864,-1.847187,-3.403764,0.242972,0.266850,3.403764,1.847187,1.556577,0.542689,2.625475,-2.860094,-2.615657,-2.849463,-2.566088
5000004.0,500.0005,-2.615657,0.00000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,-2.661265,-2.68356,-0.884686,-3.817629,0.309878,0.277272,3.817629,0.884686,2.932943,0.231737,2.351157,-2.693608,-2.690864,-1.847187,-3.403764,0.242972,0.266850,3.403764,1.847187,1.556577,0.542689,2.625475,-2.435126,-2.566088,-2.860094,-2.738020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999995.0,699.9996,-2.909200,-2.77607,0.036330,0.190604,-3.0669,-2.4672,0.0,-2.766912,0.051755,0.227497,-3.2211,-2.3514,0.0,-2.745875,0.058040,0.240915,-3.2211,-2.0735,0.0,-2.726710,0.062546,0.250091,-3.6047,-1.9711,0.425747,-2.644885,-2.69930,-0.713800,-3.672500,0.386389,0.297775,3.672500,0.713800,2.958700,0.194364,2.193150,-2.720726,-2.717650,-1.762500,-3.672500,0.251785,0.276969,3.672500,1.762500,1.910000,0.479918,2.717500,-2.828500,-2.742200,-2.737100,-2.828500
6999996.0,699.9997,-2.742200,-2.80357,0.025017,0.158168,-3.0669,-2.5609,0.0,-2.772244,0.050007,0.223622,-3.2211,-2.3514,0.0,-2.749960,0.056307,0.237291,-3.2211,-2.0735,0.0,-2.727042,0.062445,0.249890,-3.6047,-1.9711,0.527975,-2.644885,-2.69930,-0.713

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        if col != 'time':
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
dtrain = reduce_mem_usage(dtrain)
dtest = reduce_mem_usage(dtest)

Mem. usage decreased to 567.44 Mb (72.5% reduction)
Mem. usage decreased to 225.07 Mb (72.2% reduction)


In [ ]:
dtrain[dtrain.isnull().values==True]#沒有缺失值

,time,signal,open_channels,rolling_mean_10,rolling_var_10,rolling_std_10,rolling_min_10,rolling_max_10,norm_10,rolling_mean_50,rolling_var_50,rolling_std_50,rolling_min_50,rolling_max_50,norm_50,rolling_mean_100,rolling_var_100,rolling_std_100,rolling_min_100,rolling_max_100,norm_100,rolling_mean_1000,rolling_var_1000,rolling_std_1000,rolling_min_1000,rolling_max_1000,norm_1000,meanbatch,medianbatch,maxbatch,minbatch,stdbatch,mean_abs_chgbatch,abs_maxbatch,abs_minbatch,rangebatch,maxtominbatch,abs_avgbatch,meanbatch_slices2,medianbatch_slices2,maxbatch_slices2,minbatch_slices2,stdbatch_slices2,mean_abs_chgbatch_slices2,abs_maxbatch_slices2,abs_minbatch_slices2,rangebatch_slices2,maxtominbatch_slices2,abs_avgbatch_slices2,signal_shift_+1,signal_shift_-1,signal_shift_+2,signal_shift_-2


In [ ]:
y = dtrain['open_channels'].values
#除了後面寫的列以外都要
col = [c for c in dtrain.columns if c not in ['time', 'open_channels', 'group', 'medianbatch', 'abs_avgbatch', 'abs_maxbatch']]

#X=dtrain[col],y=dtrain['open_channels']
X_train, X_valid, y_train, y_valid = train_test_split(dtrain[col], dtrain['open_channels'], test_size=0.3, random_state=87)
del dtrain

In [ ]:
X_train=X_train.values
X_valid=X_valid.values

In [ ]:
y_train=y_train.values
y_valid=y_valid.values

In [ ]:
train_set = xgb.DMatrix(X_train, label=y_train,feature_names=feature_names)
val_set = xgb.DMatrix(X_valid, label=y_valid)
all_train=xgb.DMatrix(train_set,val_set)

test_set=xgb.DMatrix(dtest[col].values)

del X_train, X_valid, y_train, y_valid,dtest
gc.collect()

10

In [ ]:
%%time
params_xgb = {'colsample_bytree': 0.375,
              'learning_rate': 0.05,
              'max_depth': 10, 
              'subsample': 1, 
              'objective':'reg:squarederror',
              'eval_metric':'logloss'}

xgbmodel = xgb.train(params_xgb, all_train, num_boost_round=1000, evals=[(train_set, 'train'), (val_set, 'val')], verbose_eval=25, early_stopping_rounds=200)
del train_set, val_set
gc.collect()

joblib.dump(xgbmodel,'xgb_model.dat')

from google.colab import files
files.download("xgb_model.dat")

[0]	train-logloss:-2.09311	val-logloss:-2.08501
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 200 rounds.
[25]	train-logloss:-72.6977	val-logloss:-72.552
[50]	train-logloss:-72.7476	val-logloss:-72.6022
[75]	train-logloss:-72.7604	val-logloss:-72.6147
[100]	train-logloss:-72.7641	val-logloss:-72.6178
[125]	train-logloss:-72.7653	val-logloss:-72.6188
[150]	train-logloss:-72.7657	val-logloss:-72.6191
[175]	train-logloss:-72.7659	val-logloss:-72.6192
[200]	train-logloss:-72.766	val-logloss:-72.6193
[225]	train-logloss:-72.7662	val-logloss:-72.6193
[250]	train-logloss:-72.7663	val-logloss:-72.6193
[275]	train-logloss:-72.7664	val-logloss:-72.6194
[300]	train-logloss:-72.7665	val-logloss:-72.6194
[325]	train-logloss:-72.7666	val-logloss:-72.6194
[350]	train-logloss:-72.7667	val-logloss:-72.6194
[375]	train-logloss:-72.7668	val-logloss:-72.6194
[400]	train-logloss:-72.7669	val-logloss:-72.6195
[425]	tra

MessageError: ignored

In [ ]:
del xgbmodel

In [ ]:
dtest

In [ ]:
#記得上傳模型，上傳完別再訓練


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1idxdGKOfiJv2eqL2ejv7bTMLVK11KawU'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('xgb_model.dat')
xgbm=joblib.load('xgb_model.dat')

dMtest=xgb.DMatrix(dtest[col])

preds=xgbm.predict(dMtest)

submission = pd.DataFrame({"time":dtest['time'].values, "open_channels":preds})

# 限定預測值在0~10且整數
submission['open_channels'] = np.round(np.clip(preds, 0, 10)).astype(int)

submission[['time','open_channels']].to_csv('submission.csv', index=False, float_format='%.4f')


In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance

#才不會讓feature names變f1
#model.get_booster().feature_names = ["your", "feature", "name", "list"]
#xgboost.plot_importance(model.get_booster())

fig,ax = plt.subplots(figsize=(15,15))
plot_importance(xgbm,
                height=0.5,
                ax=ax,
                max_num_features=64)
plt.show()